In [1]:
from __future__ import print_function
import os
import sys
import cv2
from matplotlib import pyplot as plt
import numpy as np

from local_config import CURRENT_DIR
sys.path.append(os.path.join(CURRENT_DIR, ".."))

try:
    from config import Config
    from data_loader import WaymoStixelDataset
except:
    print("failed to load module")
    
from albumentations import (
    Resize,
    Compose,
    CLAHE,
    HueSaturationValue,
    RandomBrightness,
    RandomContrast,
    RandomGamma,
    ToFloat,
    Normalize,
    GaussNoise,
    RandomShadow,
    RandomRain,
)

import utility
import importlib
from models.stixel_net import build_stixel_net_inceptionV3

# TensorFlow
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    ReduceLROnPlateau,
    EarlyStopping,
)

2022-03-11 20:13:00.390383: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
dt_config = Config()
dt_config.display()


Configurations:
CURRENT_DIR                    /home/menz/StixelNet/obstacle_detection_stixelnet
DATA_PATH                      /home/menz/StixelNet/obstacle_detection_stixelnet/data/StixelNet_Waymo
GROUND_TRUTH_PATH              /home/menz/StixelNet/obstacle_detection_stixelnet/data/StixelNet_Waymo/waymo_train.txt
NUM_EPOCHS                     50
SAVED_MODELS_PATH              saved_models




In [3]:
## Train and Val set


train_aug = Compose(
    [
        GaussNoise(p=1.0),
        RandomShadow(p=0.5),
        RandomRain(p=0.5, rain_type="drizzle"),
        RandomContrast(limit=0.2, p=0.5),
        RandomGamma(gamma_limit=(80, 120), p=0.5),
        RandomBrightness(limit=0.2, p=0.5),
        HueSaturationValue(
            hue_shift_limit=5, sat_shift_limit=20, val_shift_limit=10, p=0.5
        ),
        CLAHE(p=0.5, clip_limit=2.0),
        Normalize(p=1.0),
    ]
)
    
# AUGMENTATIONS DISABLED
train_set = WaymoStixelDataset(
        data_path=dt_config.DATA_PATH,
        ground_truth_path=os.path.join(dt_config.DATA_PATH, "waymo_train.txt"),
        batch_size=8,
        transform=None,
        customized_transform=utility.HorizontalFlip(p=0.5),
    )




val_aug = Compose([Normalize(p=1.0)])
val_set = WaymoStixelDataset(
        data_path=dt_config.DATA_PATH,
        ground_truth_path=os.path.join(dt_config.DATA_PATH, "waymo_val.txt"),
        transform=None,
    )


/home/menz/miniconda3/envs/StixelNet-TF/lib/python3.9/site-packages/albumentations/augmentations/transforms.py:1826: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(
/home/menz/miniconda3/envs/StixelNet-TF/lib/python3.9/site-packages/albumentations/augmentations/transforms.py:1800: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  warnings.warn(


In [4]:
# StixelNet regression setup


model = build_stixel_net_inceptionV3()
opt = optimizers.Adam(0.01)
lossF = losses.MeanSquaredError()

model.compile(loss=lossF, optimizer=opt)
model.summary()

2022-03-11 20:13:07.502549: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-11 20:13:07.503701: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-11 20:13:07.560741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-11 20:13:07.560917: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX TITAN X computeCapability: 5.2
coreClock: 1.076GHz coreCount: 24 deviceMemorySize: 11.92GiB deviceMemoryBandwidth: 313.37GiB/s
2022-03-11 20:13:07.560942: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-03-11 20:13:07.597043: I tensorflow/stream_executor/platfor

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1280, 1920, 3)]   0         
_________________________________________________________________
resizing (Resizing)          (None, 320, 480, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 320, 480, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 320, 480, 3)       0         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 8, 13, 1536)       54336736  
_________________________________________________________________
flatten (Flatten)            (None, 159744)            0         
_________________________________________________________________
batch_normalization_203 (Bat (None, 159744)            638976

In [5]:
X,y = train_set[0]
print(np.shape(X))
print(np.shape(y))

(8, 1280, 1920, 3)
(8, 240)


In [6]:
# Training

num_epoch = 1000

callbacks = [
    ModelCheckpoint(
        os.path.join(dt_config.SAVED_MODELS_PATH, "model-{epoch:03d}-{loss:.4f}.h5"),
        monitor="val_loss",
        verbose=1,
        save_best_only=False,
        mode="auto",
        save_freq="epoch",
    ),    
    EarlyStopping(
        monitor="val_loss", min_delta=0, patience=10, verbose=0, mode="auto"
    ),
]


history = model.fit(
        train_set,       
        validation_data=val_set,      
        epochs=num_epoch,
        callbacks=callbacks,
        shuffle=True,
    )

2022-03-11 20:13:17.718468: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-11 20:13:17.739451: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3292470000 Hz


Epoch 1/1000


2022-03-11 20:13:35.199563: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-03-11 20:13:36.048588: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


 74/624 [==>...........................] - ETA: 9:05 - loss: 622850.2703

KeyboardInterrupt: 